# 上下文管理器

with 语句的目的是简化 try/finally 模式，其中 finally 语句用于执行“收拾”的操作：释放资源，还原现场等。上下文管理器协议包含`__enter__`和`__exit__`两个方法，with 语句开始和运行结束时，会分别在上下文管理器对象上调用这两个方法。

以文件对象为例：

In [1]:
with open('mirror.py') as fp:
    print(type(fp))
    src = fp.read(60)

<class '_io.TextIOWrapper'>


In [2]:
len(src)

60

with 块并没有定义新的作用域，因此 fp 对象仍然可用：

In [3]:
fp

<_io.TextIOWrapper name='mirror.py' mode='r' encoding='UTF-8'>

In [4]:
fp.closed, fp.encoding

(True, 'UTF-8')

但是并不能执行I/O操作，因为 with 语句块结束之后调用了`TextIOWrapper.__exit__`方法把文件关闭了

In [5]:
fp.read(60)

ValueError: I/O operation on closed file.

LookingGlass类实现了将标准输出转置的功能，查看细节：[LookingGlass (mirror.py)](mirror.py)。

注意查看上下文管理器和`__enter__`方法返回的对象之间的差别。

如果一切正常，在调用`__exit__`方法是传入的参数是`None`, `None`, `None`。

In [6]:
from mirror import LookingGlass
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [7]:
what

'JABBERWOCKY'

可

In [8]:
print('Back to normal')

Back to normal


传给`__exit__`方法的三个参数如下：
- exc_type
  - 异常类。如ZeroDivisionError。
- exc_value
  - 异常实例。有时会有参数传给异常构造方法，例如错误消息，这些参数可以使用exc_value.args获取。
- traceback
  - traceback对象。

也可以在 with 块之外的地方使用 LookingGlass 类

In [9]:
from mirror import LookingGlass
manager = LookingGlass()

In [10]:
manager

In [11]:
monster = manager.__enter__()

因为 jupyter notebook 的输出由其 kernel 管理，以下均采用 print 输出。

In [12]:
print(monster == 'JABBERWOCKY')

eurT


In [13]:
print(monster)

YKCOWREBBAJ


In [14]:
print(manager)

>8d6d768757f7x0 ta tcejbo ssalGgnikooL.rorrim<


In [15]:
manager.__exit__(None, None, None)

In [16]:
print(monster)

JABBERWOCKY


# contextlib模块

标准库文档：[contextlib — Utilities for with-statement contexts](https://docs.python.org/3/library/contextlib.html)

### 使用@contextmanager

定义函数时使用 yield 语句将定义体分成两部分：yield 语句前面的代码在 with 块开始时执行，后面的代码在结束时执行。

查看 [looking_glass](mirror.py) 函数。

In [17]:
from mirror import looking_glass
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [18]:
what

'JABBERWOCKY'

上文`looking_glass()`的实现有一个致命错误：如果在`with`块中抛出了异常，Python解释器会将其捕获，然后在`looking_glass`函数中的`yield`表达式中里再次抛出，但是那里没有进行错误处理，因此函数会中止，用于无法恢复原来的`sys.stdout.write`方法，导致系统处于无效状态。

一个更好的实现：[looking_glass2](mirror.py)

In [19]:
from mirror import looking_glass2
with looking_glass2() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


在上述的实现中，被装饰的函数会显式地抛出异常。